-- QUESTION 1 --

a) First 3 columns of the matrix:

In [ ]:
import numpy as np

matrix = np.loadtxt("gene_expression_matrix.txt", delimiter = " ")

print(f"First three columns:\n {matrix[:, :3]}") 

b) First 3 columns of the standardized matrix:

In [ ]:
import numpy as np

matrix = np.loadtxt("gene_expression_matrix.txt", delimiter = " ")

mean_values = np.mean(matrix, axis=0)
stdeviation_values = np.std(matrix, axis=0)

standardized_matrix = (matrix - mean_values) / stdeviation_values

print(f"First three columns of standardize matrix:\n {standardized_matrix[:, :3]}") 

c) Implementation of K-means Clustering Algorithm. Lloyd Algorithm is used which involves arbitrary assignment of k cluster centers.

In [ ]:
import numpy as np


def cosine_similarity_matrix(data, centroids):
    # Normalisation of the data and centroids is essential for cosine similarity calculation in the next step. 
    #So that we already divide both vectors to their lengths in this step.
    norm_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    norm_centroids = centroids / np.linalg.norm(centroids, axis=1, keepdims=True)

    # Then we can compute cosine similarity matrix, with dot product.
    similarity_matrix = np.dot(norm_data, norm_centroids.T)
    return similarity_matrix

def k_means(data, k, max_iterations=100):
    # This function takes the data (62x2000 matrix), how many clusters we want (k) and max iterations (cluster updates).
    # Initializes centroids randomly as in the algorithm.
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]

    prev_centroids = None
    iteration = 0

    while not np.array_equal(centroids, prev_centroids) and iteration < max_iterations:
        prev_centroids = centroids.copy()

        # Cosine similarity matrix is computed.
        similarity_matrix = cosine_similarity_matrix(data, centroids)

        # Each data point is assigned to the nearest centroid.
        labels = np.argmin(similarity_matrix, axis=1)

        # Centroids are updated.
        centroids = np.array([data[labels == i].mean(axis=0) for i in range(k)])

        iteration += 1

    
    # Squared error distortion is calculated by taking the squares of differences between each data point and its assigned centroid,
    # and then summing the squared differences along each row (axis=1). Finally, the average of these sums across all data points are
    #computed.This average gives the distortion.
    distortion = np.mean(np.sum((data - centroids[labels]) ** 2, axis=1))

    return labels, centroids, distortion


d) K-Means Algorithm is run 5 times on the given data. Square error distortion for each solution is reported.

e) For the clustering solution with the lowest squared error distortion, percentage of cancer patients without including the 62nd patient is resported. Cluster of 62nd patient is also reported.

In [ ]:
# We have a 62x2000 matrix.
# Standardized matrix should be used when using cosine similarity.

results_dict = {}
min_distortion = float('inf')
best_iteration = None

for i in range(5):
    
    labels, centroids, distortion = k_means(standardized_matrix, 2)

    # Results are printed.
    print(f"\nClustering Solution {i+1}:")
    print("Cluster labels for each person:", labels)
    print("Centroids:", centroids)
    print("Squared Error Distortion:", distortion)

    # Results are stored in a dicitonary.
    results_dict[f'Iteration {i+1}'] = {'labels': labels.tolist(),'distortion': distortion}

    # Update of minimum distortion and corresponding iteration.
    if distortion < min_distortion:
        min_distortion = distortion
        best_iteration = f'Iteration {i+1}'

best_result = results_dict[best_iteration]
print(f"\nResults for the iteration with the lowest squared error distortion ({best_iteration}):")
print("Cluster labels for each person:", best_result['labels'])
print("Squared Error Distortion:", best_result['distortion'])


cluster_0_cancer_patients = []
cluster_1_cancer_patients = []

#First 40 people are cancer patients.
for idx, x in enumerate(best_result['labels']):
    if idx <= 39 and x == 0:
        cluster_0_cancer_patients.append(x)
    elif idx <= 39 and x == 1:
        cluster_1_cancer_patients.append(x)
        
cluster_0_all = best_result['labels'][:-1].count(0)
cluster_1_all = best_result['labels'][:-1].count(1)
    

cluster_0_percentage = len(cluster_0_cancer_patients)/40 * 100
cluster_1_percentage = len(cluster_1_cancer_patients)/40 * 100

print(f"\nPercentage for cancer patients in Cluster 0 is: {cluster_0_percentage}")
print(f"Percentage for cancer patients in Cluster 1 is: {cluster_1_percentage}")
print(f"The 62nd patient was assigned in the Cluster {best_result['labels'][-1]}")

f) Based on the results in (e), the percentages of cancer patients compared to overall cancer patients for two clusters are nearly around 40% and 60%. If we can say that cluster with 40% cancer patients indicates "healthy cluster" and 60% indicates "cancer patients", we can conclude that 62nd patient might be healthy, since she/he is always assigned to the cluster with the lower cancer patient percentage. I would say it is not quite reliable (not significant) due to the "locally optimum" drawback of k-means clustering (Lloyd Algorithm). Converging in k-means doesn't mean we converge to the optimal clustering. In this manner, the seeds we start come into question. Seeds can be chosen other than randomizing method such as using some heuristic or using hierarchical clustering to find good seeds.